# The Pop2net basics

After the installation of Pop2net, the first thing we have to do is to import the package in order to use it:

In [1]:
import pop2net as p2n

In [2]:
# this is a hidden cell which sets seeds for consistent outputs

import random

import numpy as np

random.seed(0)
np.random.seed(0)

## Model

In Pop2net, `Model` is the most important object class because almost everything happens inside - or at least in regard to - a `Model`.
You can imagine a `Model` as the world where all the entities we create live.
Therefore, everthing you can do in Pop2net starts with creating an instance of `p2n.Model`.

In [3]:
model = p2n.Model()

A `Model` has three very important attributes:

1. `model.g` is a network-graph which stores all agents, locations and their relations to each other as a bipartite network:

In [4]:
model.g

You never need to access `model.g` yourself unless you are a very experienced Pop2net user.
In most cases, anything you want to do with the network can be done using the convenient methods provided by Pop2net.

2. `model.agents` returns a list of all agents stored in `model.g`:

In [5]:
model.agents

AgentList (0 objects)

3. `Model.locations` returns a list of all locations stored in `Model.g`:

In [6]:
model.locations

LocationList (0 objects)

## Agents and locations

Next, we create our first agent:

In [7]:
agent = p2n.Agent(model)

Whenever a new instance of an `Agent` is created, it gets automatically added to the given instance of `Model`.
This means that the created `Agent` instance is represented as a node in `model.g` ...

In [8]:
model.g.nodes

NodeView((1,))

... and gets returned in `model.agents`:

In [9]:
model.agents

AgentList (1 object)

The same applies to locations:

In [10]:
location = p2n.Location(model)

In [11]:
model.g.nodes

NodeView((1, 2))

In [12]:
model.locations

LocationList (1 object)

## A quick look: Plotting a network graph

Often the best thing you can do to understand things is to look at them.
In Pop2net, the `NetworkInspector` helps you to understand the network you are building.
For instance, using the `NetworkInsepctor`, you can visualize the bipartite network of agents and locations:

In [13]:
inspector = p2n.NetworkInspector(model)
inspector.plot_bipartite_network()

To be honest, the above is not really a network because there are only two nodes without any edge.
However, the graph gives an intuitive overview of the entities in your model.

There is also a method to plot the graph at the agent level:

In [14]:
inspector.plot_agent_network()

And there is also a method to plot both graphs!

In [15]:
inspector.plot_networks()

## Creating edges between agents and locations

Our model has one agent and one location.
Let's create an edge between those two entities by *adding* the agent to the location:

In [16]:
location.add_agent(agent)

By the way: we could also let the `agent` do the same action by running `agent.add_location(location)`.
And we could also let the model do the work using `model.add_agent_to_location(location=location, agent=agent)`.

Anyway, let's have a look at the network graph:

In [17]:
inspector.plot_bipartite_network()

The attribute `location.agents` shows all agents that are connected to this location:

In [18]:
location.agents

AgentList (1 object)

The attribute agent.locations shows all locations that are connected to the agent:

In [19]:
agent.locations

LocationList (1 object)

Let's add 3 more agents to the model using a For-Loop:

In [20]:
for _ in range(3):
    p2n.Agent(model)

As you can see, we do not even have to assign the created instances of `Agent` to a variable because their are added to the model automatically during the initialization.
Now we have 4 agents in our model:

In [21]:
model.agents

AgentList (4 objects)

Let's add an additional location.
But now we create our own location class `School` which inherits from `p2n.Location`.
You will understand the purpose of this later.

In [22]:
class School(p2n.Location):
    pass

Again, we only have to initialize the instance of `School` to add it to the model:

In [23]:
School(model)

School (Obj 6)

Now, the model has two locations:

In [24]:
model.locations

LocationList (2 objects)

Let's have a look at the bipartite network graph:

In [25]:
inspector.plot_bipartite_network()

Because there is still only one edge, in the next step, we assign each agent to one location and make sure that each location has not more than 2 agents:

In [26]:
for location in model.locations:
    for agent in model.agents:
        if len(location.agents) < 2 and len(agent.locations) == 0:
            location.add_agent(agent)

Let's look at the graph again:

In [27]:
inspector.plot_bipartite_network()

Just because we can, let's create some more edges by adding one agent to all locations:

In [28]:
agent = model.agents[0]
for location in model.locations:
    location.add_agent(agent)

In [29]:
inspector.plot_bipartite_network()

Here is the corresponding agent-level network graph:

In [30]:
inspector.plot_agent_network()

## Removing entities from the graph

So far, we only added agents to the model, locations to the model, agents to locations and locations to agents.
Of course we can also remove things.

To remove the edge between an agent and a location, we can use `mode.remove_agent_from_location()`, `agent.remove_location()` and `location.remove_agent()`.
In the following example, we remove the edge between `agent` and `location` using `location.remove_agent()`:

In [31]:
location.remove_agent(agent)

In [32]:
inspector.plot_networks()

To remove agents or locations from the graph, we can use `model.remove_agent()` and `model.remove_location()`.
In the following example, we remove `agent` from the graph:

In [33]:
model.remove_agent(agent)

In [34]:
inspector.plot_networks()

Let's put `agent` back in the graph:

In [35]:
model.add_agent(agent)

In [36]:
inspector.plot_networks()

## Connecting agents quickly

In Pop2net, agents are considered connected if they share a location.
Thus, in order to connect agents, we first have to create a location first and then assign the agents to this location.

The methods `agent.connect()` and `model.connect_agents()` simplify that process by creating an instance from a given location class and directly assigning the given agents to this location instance.
In the following, we connect `agent` with the second agent of the model `model.agents[1]` using `agent.connect()`:

In [37]:
agent.connect(agent=model.agents[1], location_cls=p2n.Location)

In [38]:
inspector.plot_networks()

Using `model.connect_agents()` we can also connect multiple agents at the same time.
In addition, this time we use a custom location class to connect the agents:

In [39]:
class World(p2n.Location):
    pass

In [40]:
model.connect_agents(agents=model.agents, location_cls=World)

In [41]:
inspector.plot_networks()

## Finding neighbors and shared locations

From the perspective of an agent, it is very important to know with whom the agent is connected in the network.
As Pop2net works with bipartite networks, technically agents are only connected with locations.
As you might already know, agents can access all connected locations via `agent.locations`:

In [42]:
agent.locations

LocationList (2 objects)

As locations are mainly a helping tool to connect agents, we often want to know which other agents are connected to the same locations.
Every agent can access those other agents who are at the same location via `agent.neighbors()`:

In [43]:
agent.neighbors()

AgentList (3 objects)

If we want to find only those *neighbors* who share a specific type of location with the agent, we can use the argument `location_classes` to filter by types of locations.
In the following we only want *neighbors* who share a location of the type `School` with the agent:

In [44]:
agent.neighbors(location_classes=[School])

AgentList (0 objects)

By using different types of locations we can organize the relations of an agent to other agents and then access *neighbors* with a certain relation to the agent as we just did it above.

Sometimes we need to access all the locations an agent shares with another agent.
This can be done using `agent.shared_locations()`:

In [45]:
agent2 = model.agents[2]

agent.shared_locations(agent2)

LocationList (1 object)

In [46]:
agent.shared_locations(agent2)[0]

World (Obj 8)

`Agent.shared_locations()` also has the argument `location_classes` to filter by the type of the location:

In [47]:
agent.shared_locations(agent2, location_classes=[p2n.Location])

LocationList (0 objects)

In [48]:
agent.shared_locations(agent2, location_classes=[World])

LocationList (1 object)